In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

import re
import pickle


In [ ]:
df = pd.read_csv('D:\UNI\CLASE MLOPS\data\base_encuestados_v2.csv')

In [3]:
df.head()

,id_persona_token,periodo_encuesta,Ramo_encuesta,Seg_Ramo_encuesta,Siniestralidad_encuesta,Cod_Producto_encuesta,Producto_encuesta,Comentarios,Categoria,NPS
0,cNS84LyBtd7sSQtborT2tEwMytv9qku3M1x0+n9L0v0=,2022-01-01 00:00:00,RRGG,RRGG - DOMI,No Siniestrado,1403,BANCO CONTINENTAL (CONTICASA),PORQUE LLAME A RIMAC POR UNA EMERGENCIA DE SAL...,0,DETRACTOR
1,UKZkSIR3QIDK86p60yURPXw+8VwbXJp5wZ9iduk07Ec=,2022-02-01 00:00:00,VEHICULAR,VEHICULAR - VEHICULAR,Siniestrado,2001,VEH?CULOS,PORQUE RIMAC ME MANDÓ UNA CARTA NOTARIAL A MI...,0,DETRACTOR
2,c3sTyiA4gnAi8LliSGET9oOTPudIHFyEBcBScSro0oI=,2022-02-01 00:00:00,VEHICULAR,VEHICULAR - VEHICULAR,Siniestrado,2001,VEH?CULOS,PORQUE LLEGUE AL TALLER COMO ME DIJERON PORQUE...,0,DETRACTOR
3,BY5oTjJmI/vZqJYIEs14nwxlI97Hm+tUn0Mmp292DH0=,2022-03-01 00:00:00,VEHICULAR,VEHICULAR - VEHICULAR,Siniestrado,2001,VEH?CULOS,PORQUE SE DEMORARON 2 HORAS Y CUARTO PARA ASIS...,0,DETRACTOR
4,JwFc5A+aHZNFVohxzaA4TTphFM9Cbz+b2F0w0g92eUw=,2022-01-01 00:00:00,VEHICULAR,VEHICULAR - VEHICULAR,Siniestrado,2001,VEH?CULOS,QUISE HACER QUE ME ATIENDAN EL SEGURO PORQUE H...,0,DETRACTOR


In [4]:
df = df[['Comentarios','NPS']].dropna().copy()

In [5]:
df['Comentarios'] = df['Comentarios'].apply(lambda x: x.lower())
df['Comentarios'] = df['Comentarios'].apply(lambda x: re.sub(r'[^a-zA-z0-9\s]', '', x))

In [6]:
le = LabelEncoder()
df['NPS_encoded'] = le.fit_transform(df['NPS'])

# integer labels for modeling
y = df['NPS_encoded'].values

In [7]:
max_features = 1000
tokenizer = Tokenizer(num_words=max_features, split = ' ')
tokenizer.fit_on_texts(df['Comentarios'].values)
X = tokenizer.texts_to_sequences(df['Comentarios'].values)
X = pad_sequences(X)
X.shape

(24929, 922)

In [8]:
emdeb_dim = 50
model = Sequential()
model.add(Embedding(max_features, emdeb_dim, input_length = X.shape[1]))
model.add(LSTM(10))
model.add(Dense(len(df['NPS_encoded'].unique()), activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

c:\Users\Fabrizio\scoop\apps\python\3.10.7\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
y = pd.get_dummies(df['NPS_encoded']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(19943, 922) (19943, 3)
(4986, 922) (4986, 3)


In [10]:
model.fit(X_train, y_train, epochs=5, verbose=1, validation_data=(X_test, y_test))

Epoch 1/5
624/624 ━━━━━━━━━━━━━━━━━━━━ 75s 118ms/step - accuracy: 0.7860 - loss: 0.5387 - val_accuracy: 0.8510 - val_loss: 0.3922
Epoch 2/5
624/624 ━━━━━━━━━━━━━━━━━━━━ 73s 117ms/step - accuracy: 0.8550 - loss: 0.3643 - val_accuracy: 0.8592 - val_loss: 0.3510
Epoch 3/5
624/624 ━━━━━━━━━━━━━━━━━━━━ 74s 118ms/step - accuracy: 0.8697 - loss: 0.3211 - val_accuracy: 0.8181 - val_loss: 0.3995
Epoch 4/5
624/624 ━━━━━━━━━━━━━━━━━━━━ 79s 126ms/step - accuracy: 0.8772 - loss: 0.3045 - val_accuracy: 0.8488 - val_loss: 0.3477
Epoch 5/5
624/624 ━━━━━━━━━━━━━━━━━━━━ 85s 136ms/step - accuracy: 0.8860 - loss: 0.2780 - val_accuracy: 0.8622 - val_loss: 0.3414


In [11]:
test = ['El servicio fue excelente y muy rápido']
test = tokenizer.texts_to_sequences(test)
test = pad_sequences(test, maxlen=X.shape[1], dtype='int32', value=0)
print(model.predict(test))
sentiment = model.predict(test)[0]
if(np.argmax(sentiment) == 0):
    print("Detractor")
elif (np.argmax(sentiment) == 1):
    print("Pasivo")
else:
    print("Promotor")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
[[0.01571313 0.02833439 0.95595247]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Promotor


In [ ]:
with open('models/tokenizer.pickle', 'wb') as tk:
    pickle.dump(tokenizer, tk, protocol=pickle.HIGHEST_PROTOCOL)

model_json = model.to_json()
with open("models/model.json", "w") as js:
    js.write(model_json)

model.save_weights('models/.model.weights.h5')